In [12]:
import gym
import random
import numpy as np
import tflearn
from tflearn.layers.core import input_data, dropout, fully_connected
from tflearn.layers.estimator import regression
from statistics import median, mean
from collections import Counter

In [13]:
LR = 1e-3
env = gym.make('CartPole-v0')
env.reset()
goal_steps = 500
score_requirement = 50
initial_games = 10000

In [14]:
def some_random_games_first():
    for episode in range(5):
        env.reset()
        for t in range(goal_steps):
            env.render()
            action = env.action_space.sample()
            observation, reward, done, info = env.step(action)
            if done:
                break

In [15]:
#some_random_games_first()

In [16]:
def initial_population():
    training_data = []
    scores = []
    accepted_scores = []
    for _ in range(initial_games):
        score= 0
        game_memory = []
        prev_observation = []
        for _ in range(goal_steps):
            action = random.randrange(0,2)
            observation, reward, done, info = env.step(action)
            
            if len(prev_observation) > 0:
                game_memory.append([prev_observation, action])
                
            prev_observation = observation
            score += reward
            if done:
                break

        if score >= score_requirement:
            accepted_scores.append(score)
            for data in game_memory:
                if data[1] == 1:
                    output = [0, 1]
                elif data[1] == 0:
                    output = [1, 0]
                
                training_data.append([data[0], output])
        env.reset()
        scores.append(score)
        
    training_data_save = np.array(training_data)
    np.save("saved.npy",training_data_save)
    
    print('Average accepted score:', mean(accepted_scores))
    print('Median accepted score:', median(accepted_scores))
    print(Counter(accepted_scores))

    return training_data

In [17]:
training_data = initial_population()

Average accepted score: 61.621776504298
Median accepted score: 58.0
Counter({50.0: 33, 51.0: 28, 56.0: 25, 53.0: 22, 52.0: 20, 55.0: 19, 59.0: 16, 61.0: 15, 54.0: 15, 70.0: 13, 58.0: 13, 57.0: 10, 62.0: 9, 63.0: 8, 68.0: 8, 73.0: 7, 66.0: 7, 64.0: 7, 60.0: 6, 65.0: 5, 67.0: 5, 76.0: 5, 69.0: 5, 71.0: 4, 75.0: 4, 87.0: 3, 74.0: 3, 72.0: 3, 81.0: 3, 77.0: 3, 90.0: 2, 80.0: 2, 85.0: 2, 99.0: 2, 95.0: 2, 92.0: 2, 97.0: 2, 91.0: 1, 108.0: 1, 98.0: 1, 88.0: 1, 84.0: 1, 96.0: 1, 112.0: 1, 94.0: 1, 104.0: 1, 116.0: 1, 78.0: 1})


In [18]:
def neural_network_model(input_size):
    
    network = input_data(shape=[None, input_size, 1], name = "input")
    
    network = fully_connected(network, 128, activation = "relu")
    network = dropout(network, 0.8)
    
    network = fully_connected(network, 256, activation = "relu")
    network = dropout(network, 0.8)
    
    network = fully_connected(network, 512, activation = "relu")
    network = dropout(network, 0.8)
    
    network = fully_connected(network, 256, activation = "relu")
    network = dropout(network, 0.8)
    
    network = fully_connected(network, 128, activation = "relu")
    network = dropout(network, 0.8)
    
    network = fully_connected(network, 2, activation = "softmax")
    network = regression(network, optimizer = 'adam', learning_rate = LR,
                         loss = 'categorical_crossentropy', name = "targets")
    
    model = tflearn.DNN(network, tensorboard_dir = 'log')
    
    return model

In [19]:
def train_model(training_data, model=False):

    X = np.array([i[0] for i in training_data]).reshape(-1,len(training_data[0][0]),1)
    y = [i[1] for i in training_data]

    if not model:
        model = neural_network_model(input_size = len(X[0]))
    
    model.fit({'input': X}, {'targets': y}, n_epoch=3, snapshot_step=500, show_metric=True, run_id='openai_learning')
    return model

In [20]:
model = train_model(training_data)

---------------------------------
Run id: openai_learning
Log directory: log/
INFO:tensorflow:Summary name Accuracy/Adam_0 (raw) is illegal; using Accuracy/Adam_0__raw_ instead.
INFO:tensorflow:Summary name Accuracy_1/Adam_1 (raw) is illegal; using Accuracy_1/Adam_1__raw_ instead.
---------------------------------
Training samples: 42314
Validation samples: 0
--


InvalidArgumentError: You must feed a value for placeholder tensor 'input_1/X' with dtype float and shape [?,4,1]
	 [[node input_1/X (defined at /anaconda3/lib/python3.6/site-packages/tflearn/layers/core.py:81)  = Placeholder[dtype=DT_FLOAT, shape=[?,4,1], _device="/job:localhost/replica:0/task:0/device:CPU:0"]()]]

Caused by op 'input_1/X', defined at:
  File "/anaconda3/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/anaconda3/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/anaconda3/lib/python3.6/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/anaconda3/lib/python3.6/site-packages/ipykernel/kernelapp.py", line 505, in start
    self.io_loop.start()
  File "/anaconda3/lib/python3.6/site-packages/tornado/platform/asyncio.py", line 1

In [ ]:
scores = []
choices = []

for each_game in range(100):
    score = 0
    game_memory = []
    prev_obs = []
    env.reset()
    for _ in range(goal_steps):
        #env.render()
        if len(prev_obs) == 0:
            action = random.randrange(0,2)
        else:
            action = np.argmax(model.predict(prev_obs.reshape(-1,len(prev_obs),1))[0])

        choices.append(action)
                
        new_observation, reward, done, info = env.step(action)
        prev_obs = new_observation
        game_memory.append([new_observation, action])
        score+=reward
        if done: break

    scores.append(score)

print('Average Score:',sum(scores)/len(scores))
print('choice 1:{}  choice 0:{}'.format(choices.count(1)/len(choices),choices.count(0)/len(choices)))
print(score_requirement)